In [1]:
from core.request import FactoryClient


In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm

Connected to /address namespace!
Connected to /address namespace!


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
test_address = '0x7e5ce10826ee167de897d262fcc9976f609ecd2b'



In [ ]:

# result = f.sync_request(
#     namespace='/address',
#     payload={
#         'address': test_address,
#         'currency': 'usd',
#         'portfolio_fields': 'all',
# #         'transactions_offset': 2
#     },
#     scope='transactions'
# )

In [8]:
def parce_transactions(wallet_hash):
    f = FactoryClient()
    dicts = []
    i = 50
    data = f.sync_request(
        namespace='/address',
        payload={
            'address': wallet_hash,
            'currency': 'usd',
            'portfolio_fields': 'all',
            'transactions_offset': 0
        },
        scope='transactions'
    )
    dicts.append(data)

    while len(data) > 0:
        f = FactoryClient()
        data = f.sync_request(
                                namespace='/address',
                                payload={
                                    'address': test_address,
                                    'currency': 'usd',
                                    'portfolio_fields': 'all',
                                    'transactions_offset': i
                                        },
                                scope='transactions'
                                )
        i += 50
        dicts.append(data)
    return dicts

In [9]:
dicts = parce_transactions(test_address)

Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is received
Connected to /address namespace!
Address portfolio is r

In [31]:
dicts[0][0]

{'id': '827f8b05-fb664ffdaac7101f-7e5ce108',
 'type': 'send',
 'protocol': None,
 'mined_at': 1623409428,
 'block_number': 12612869,
 'status': 'confirmed',
 'hash': '0xfb664ffdaac7101f2b751eb5e3c45e60a4dc52431d9f79cecb85c4c2b7781ff2',
 'direction': 'out',
 'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
 'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',
 'contract': None,
 'nonce': 660,
 'changes': [{'asset': {'asset_code': 'eth',
    'name': 'Ethereum',
    'symbol': 'ETH',
    'decimals': 18,
    'type': None,
    'icon_url': 'https://s3.amazonaws.com/token-icons/eth.png',
    'price': {'value': 2408.05,
     'relative_change_24h': -0.843308503497997,
     'changed_at': 0},
    'is_displayable': True,
    'is_verified': True},
   'value': 500000000000000000,
   'direction': 'out',
   'address_from': '0x7e5ce10826ee167de897d262fcc9976f609ecd2b',
   'address_to': '0x51020400c2aa3ed9856435716c50b6ef2f38e084',
   'price': 2485.25}],
 'fee': {'value': 105000000000

In [49]:
def get_price(changes_dict, deg=18):
    stock_price = changes_dict['price']
    value = changes_dict['value']
    if (stock_price is None) or (value is None) or (deg is None):
        return np.nan
    order_price = (value / 10**deg) * stock_price
    return order_price
    
def parse_changes(transaction, type_of_deal, mined_at, hash_tr):
    changes = transaction['changes']
    if type_of_deal == 'send':
        x = changes[0]
        deg = x['asset']['decimals']
        outcome = get_price(x, deg)
        income = 0
    
    elif type_of_deal == 'receive':
        x = changes[0]
        deg = x['asset']['decimals']
        outcome = 0
        income = get_price(x, deg)
    
    elif type_of_deal == 'trade':
        income = np.nan
        outcome = np.nan
        if len(changes) == 0:
            income = 0
            outcome = 0
        if len(changes) == 1:
            x = changes[0]
            deg_x = x['asset']['decimals']
            if x['direction'] == 'in':
                income = get_price(x, deg_x)
            else:
                outcome = get_price(x, deg_x)
             
        if len(changes) == 2:
            x, y = changes[0], changes[1]
            deg_x = x['asset']['decimals']
            deg_y = y['asset']['decimals']
            
            if x['direction'] == y['direction']:
                if x['direction'] == 'in':
                    income = get_price(x, deg_x)
                    outcome = 0
                else:
                    income = 0
                    outcome = get_price(x, deg_x)
            else:
                
                if x['direction'] == 'in':
                    income = get_price(x, deg_x)
                else:
                    outcome = get_price(x, deg_x)
                if y['direction'] == 'in':
                    income = get_price(y, deg_y)
                else:
                    outcome = get_price(y, deg_y)
                    
    elif type_of_deal == 'claim':
        x = changes[0]
        deg = x['asset']['decimals']
        outcome = 0
        income = get_price(x, deg)
        
    elif type_of_deal == 'authorize':
        outcome = 0
        income = 0
        
    fee = transaction['fee']
    if fee is not None:
        fee_price = get_price(fee)
    else:
        fee_price = np.nan
    df = pd.DataFrame(data={'timestamp': mined_at, 'type': type_of_deal, 'income': income, 'outcome': outcome, 'fee': fee_price}, index=[hash_tr])
    return df


In [50]:
dfsss = []
for i, data in enumerate(dicts[:-1]):
    print(i)
    dfs = []
    for res in tqdm(data):
        if res['type'] in ['send', 'receive', 'trade', 'claim', 'authorize']:
            chank = parse_changes(res, res['type'], res['mined_at'], res['hash'])
            dfs.append(chank)
        else:
            print(res['type'])
    dfsss.append(pd.concat(dfs))

100%|██████████| 50/50 [00:00<00:00, 2523.28it/s]

0
execution
execution
execution
execution
execution
execution
execution
execution
execution
1
execution
execution
execution
execution
stake
unstake
2
stake
stake
execution
execution
3
stake
deployment
4
unstake
execution
execution
execution
execution
stake
5
unstake
stake
unstake
unstake
stake
withdraw
execution



100%|██████████| 50/50 [00:00<00:00, 1731.34it/s]


6
execution
execution
stake
execution
execution
7
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
withdraw
withdraw
8
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
withdraw
withdraw
deposit
deposit
deposit
deposit
deposit
deposit
withdraw
execution
execution
execution
execution
execution
execution
deposit
withdraw
9
withdraw
execution
execution
withdraw
withdraw
deposit
withdraw
withdraw
deposit
withdraw
deposit
withdraw
deposit
withdraw
repay
10
deposit
withdraw
withdraw
withdraw
deposit
deposit
borrow
withdraw
execution
repay
borrow
deposit
repay
withdraw
withdraw
withdraw
borrow
deposit
withdraw
execution
withdraw
deposit
withdraw
deposit
11
deposit
withdraw
borrow
withdraw
deposit
deposit
execution
execution


100%|██████████| 28/28 [00:00<00:00, 2382.35it/s]


12
repay
execution
execution
repay
borrow
borrow
13
execution
execution
execution
execution
repay
14
execution
deployment
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
execution
15
deployment
execution
execution
execution
execution
execution
execution
deployment


In [51]:
pd.concat(dfsss)[0:50]

,timestamp,type,income,outcome,fee
0xfb664ffdaac7101f2b751eb5e3c45e60a4dc52431d9f79cecb85c4c2b7781ff2,1623409428,send,0.000000,1242.625000,2.609512
0xf5ca1d47c7cedd8cdd6dd7ca221924c6230576cfb9a689d640e9cdc736a57db0,1623405033,send,0.000000,246.941377,4.127481
0x41411bca53285eab62a5a03fdc2d7d180586dd3658fca1a75611faea3fd94f0b,1623404930,authorize,0.000000,0.000000,1.446793
0x669a5ea0e08a380688c2b1f455c3d9533de5a74277764921d96b2ae51edc0f29,1623404837,trade,247.182101,247.466000,4.335506
0x9ddff2f4c51808d6a1da4d754b28406f5e98603a21bf06f89f26996f43e41c9d,1622627442,trade,4126.018883,4126.018883,0.959009
0x8ef1b4a80ed452dea16264a23acddcae808931eb15aac98300cdde397a853b7e,1622627403,authorize,0.000000,0.000000,2.599983
0x4434109bab66c81bde5588b102be850f1a6ceee9a3ad14f5a91ff37b84be1ad3,1622039428,receive,8315.475458,0.000000,14.832865
0xcc3b3f1e25b971cb05d1908e9d38a053d05d82fde07c6d3b08a3243cacfca43f,1621114628,receive,NaN,0.000000,NaN
0xdf07d509ecce78bbe85d4ec6fe52b6428030e951d83130374373554ebd0f8a61,1620998825,claim,1489.258239,0.000000,25.235842
0xb0f93dca3b00537453141f6948dbd87a37a05474fb210d247dfb67c7eb149674,1620812923,send,0.000000,4310.320000,43.623313


Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!
Connected to /address namespace!


In [ ]:


"deposit"

"withdraw"

"borrow"

"repay"

"stake"

"unstake"

